# Deriving Term Frequency from CWB Weather Summary

這篇筆記要示範自然語言處理（[Natural Language Processing](https://en.wikipedia.org/wiki/Natural_language_processing), NLP）中的基本工具：計算 [term frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)。我們想要從氣象局每天的[「天氣概況」](https://www.cwb.gov.tw/V8/C/W/index.html)裡，找出氣象局專家群常常提及的一些台灣及鄰近區域的天氣現象。

以下為氣象局天氣概況的一則範例：

***
```
07fW01136
中央氣象局氣象報告
１１０年３月１３日１１時０分發布
３月１３日８時天氣概況：
一、高氣壓１０２６百帕，在北緯３２度，東經１２０度，即在江蘇，向東緩慢移動。
二、今、明（１３日、１４日）兩天東北季風影響，北部及東北部天氣較涼，其他地區早晚亦涼；臺灣東半部地區有局部短暫雨，其他地區及澎湖、金門、馬祖為多雲到晴，今日午後新竹以南山區亦有局部短暫陣雨，明日新竹以南山區亦有零星短暫雨；今、明兩天桃園至臺南、東南部（含蘭嶼、綠島）、恆春半島沿海空曠地區及澎湖、金門易有較強陣風；明日西半部地區易有局部霧或低雲影響能見度，請注意。
三、海上強風特報：
１、東北風偏強，臺灣海峽北部平均風力６至７級，最大陣風９級；臺灣北部海面、臺灣東南部海面及臺灣海峽南部平均風力可達６級，最大陣風８級，船隻請特別注意。今（１３）日臺灣東南部海面及臺灣海峽平均風力將稍減弱。明（１４日）下午起臺灣東南部海面平均風力將增強至６級，最大陣風８級，船隻請注意。
２、東北風偏強，巴士海峽、廣東海面及南海平均風力可達６級，雷雨區最大陣風８至９級，船隻請注意。今（１３日）晚起中西沙島海面平均風力將增強至６到７級，最大陣風９級，船隻請特別注意。今（１３）日廣東海面平均風力將稍減弱。

```
***

我們希望能從內容裡中找出像是「高氣壓」、「東北季風」、「局部霧」、「低雲」、以及「強風特報」、「颱風警報」這類指涉特定天氣系統或現象的詞彙，並透過分析長時間累積的文本資料，計算各個詞彙出現的頻率，來看看中央氣象局最常提到的天氣現象有哪些。



## 工具程式碼

這個工具的開發過程詳述在 [Deriving Term Frequency from CWB Weather Summary](https://github.com/tingsyo/data_science_practioner_notes/blob/master/notebook/NLP_term_frequency_from%20CWB_weather_summary.ipynb) 這篇筆記裡，以下我們直接看測試調整過後的結果。

In [1]:
# 讀取單一檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='utf8') as f:
        try:
            text = f.readlines()
        except UnicodeDecodeError as e:
            print(furi)
    text = ''.join(text)
    return(text)

# 利用 jieba 分詞
def calculate_tf_with_jieba(article, userdict, char_to_remove):
    import jieba
    jieba.load_userdict(userdict)    # Load user-dictionary
    # Perform word segmentation
    seg_list = jieba.cut(article, cut_all=True)
    # Count word frequency
    wc={}
    for w in list(seg_list):
        if not w in wc:
            wc[w] = 1
        else:
            wc[w] += 1
    # Remove keys containing character to remove
    for term in char_to_remove:
        for k in wc.copy().keys():
            if term in k:
                del wc[k]
    # Done
    return(wc)

# Merge two dicts
def merge_dictionary(dict1, dict2):
    newdict = dict1.copy()
    for k, v in dict2.items():
        #print(k,v)
        if k not in newdict.keys():
            newdict[k]=v
        else:
            newdict[k]+=v
    return(newdict)

# 
def count_words(DATAPATH, SURFIX = '_1100.W01.dat',
                userdict='../data/cwb/userdict.txt', 
                ignorelist='../data/cwb/ignore.txt',
                min_count=1, min_length=1):
    import os, re
    import pandas as pd
    import jieba
    from collections import Counter
    # Get files
    fileinfo = []
    for root, dirs, files in os.walk(DATAPATH):
        for name in files:
            if name.endswith(SURFIX):
                date = name.replace(SURFIX, '')
                furi = (os.path.join(root, name))
                fileinfo.append({'date':date, 'uri': furi})
    fileinfo = pd.DataFrame(fileinfo)
    #print(fileinfo.head())
    # Define characters to remove
    char_to_remove = ['１', '２', '３', '４', '５', '６', '７', '８', '９', '０', 
                      '，', '。', '、', '(', ')', '（', '）', '；', '\n', '\u3000','07fW']
    # Perform word segmentation and counts
    data = read_cwb_summary(fileinfo['uri'].iloc[0])
    wc = calculate_tf_with_jieba(data, userdict, char_to_remove)
    for i in range(1,fileinfo.shape[0]):
        data = read_cwb_summary(fileinfo['uri'].iloc[i])
        tmp = calculate_tf_with_jieba(data, userdict, char_to_remove)
        wc = merge_dictionary(wc, tmp)
    # Load ignore-list
    with open(ignorelist, 'r', encoding='utf8') as f:
        to_ignore = f.read().split('\n')
    to_ignore.append('\n')
    to_ignore.append('\u3000')
    # Remove terms in ignore list
    for term in to_ignore:
        if term in wc.copy().keys():
            del wc[term]
    # Remove terms with length less than min_mlength
    if min_length>1:
        for k in wc.copy().keys():
            if len(k)<min_length:
                del wc[k]
    if min_count>1:
        for k in wc.copy().keys():
            if wc[k]<min_count:
                del wc[k]
    # Sort by counts
    wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    # Done
    return(wc)

In [2]:
wc = count_words('D:/data/cwb_report', min_count=20, min_length=2)
print(wc)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tsyo\AppData\Local\Temp\jieba.cache
Loading model cost 0.557 seconds.
Prefix dict has been built successfully.


{'最大陣風': 18827, '雷雨': 8242, '雷雨區': 6442, '東北風': 2846, '鋒面': 2812, '低氣壓': 1642, '雷陣雨': 1493, '東北季風': 1389, '大雨': 1256, '強陣風': 993, '熱帶性低氣壓': 924, '豪雨': 289, '華南雲雨區': 158, '寒流': 142, '太平': 120, '西南氣流': 108, '拉西': 68, '溫帶氣旋': 57, '降雨': 49, '威克': 36, '安全': 35, '注意安全': 31, '天寒': 30, '雅浦': 21, '哈隆': 20}


In [14]:
import pandas as pd

targets = ['雷雨', '雷陣雨', '降雨', '大雨', '豪雨', '東北季風', '寒流', 
           '鋒面', '低氣壓','熱帶性低氣壓', '華南雲雨區', '西南氣流', '溫帶氣旋', '強陣風']

wc_selected = { k: wc[k] for k in targets }
wc_selected['year'] = 'sum'

data = []
for year in range(2012, 2021):
    print(year)
    tmp = count_words('D:/data/cwb_report/'+str(year), min_count=1, min_length=2)
    tmp_selected = {'year': year}
    for k in targets:
        try:
            tmp_selected[k] = tmp[k]
        except KeyError:
            #print('No '+k)
            tmp_selected[k] = 0
    #print(tmp_selected)
    data.append(tmp_selected)
    
data.append(wc_selected)
data = pd.DataFrame(data)
print(data)




2012
2013
2014
2015
2016
2017
2018
2019
2020
   year    雷雨   雷陣雨  降雨    大雨   豪雨  東北季風   寒流    鋒面   低氣壓  熱帶性低氣壓  華南雲雨區  \
0  2012   505   148   0    79    7   323    3   282   160      76      4   
1  2013   692   160   1    99   16   374   17   304   190     114     19   
2  2014   963   178   0    80   16    58   26   372   189      90     16   
3  2015   788    91   0    61   10    44   16   245    80      45     12   
4  2016  1168   182   1   147   23    79   25   352   200     117     24   
5  2017  1007   179  10   127   44    95    7   361   243     155     24   
6  2018  1086   188   8   190   55   120   27   281   261     185     14   
7  2019  1120   149  17   204   59   160    0   318   171      77     24   
8  2020   913   218  12   269   59   136   21   297   148      65     21   
9   sum  8242  1493  49  1256  289  1389  142  2812  1642     924    158   

   西南氣流  溫帶氣旋  強陣風  
0    10     6   47  
1     0     6   76  
2     9     9   88  
3    13     1   52  
4     6     5

In [15]:
data.to_csv('../data/cwb_report_keyword_by_year.csv', index=False)